# Project Template - adapted

## Imports

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

Be sure to start the stream on Kafka!

## Create Spark Session

In [2]:
spark = (
    SparkSession.builder
        .appName("Project DPA")
        .master("local[*]")   
        .config("spark.jars.packages",
                "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0")
        .config("spark.sql.repl.eagerEval.enabled", True)
        .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1502e608-33f5-45c0-83fc-a38090703ebd;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.0 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 164ms :: artifacts dl 5m

## Schema Definiton

In [3]:
schema = StructType([
    StructField("name", StringType()),
    StructField("price", FloatType()),
    StructField("n_shares", IntegerType()),
    StructField("timestamp", StringType())
])

# Kafka Stream Ingestion

In [4]:
# Read from Kafka topic "stock"
raw_df = (
    spark.readStream
         .format("kafka")
         .option("kafka.bootstrap.servers", "kafka1:9092")
         .option("subscribe", "stock")
         .option("startingOffsets", "latest")
         .load()
)

# Kafka sends bytes → convert to string
json_df = raw_df.selectExpr("CAST(value AS STRING)")

# Parse JSON using the schema
parsed_df = json_df.select(
    from_json(col("value"), schema).alias("data")
).select("data.*")

# Convert timestamp string into actual timestamp type
parsed_df = parsed_df.withColumn(
    "timestamp", to_timestamp("timestamp", "yyyy-MM-dd'T'HH:mm:ss'Z'")
)

# Make the dataset available to all following tasks
parsed_df.createOrReplaceTempView("stocks_stream")

# Start Streaming Output

This is optional but just to make sure that Spark is actually receiving data

In [ ]:
query = (
    parsed_df.writeStream
             .format("console")
             .outputMode("append")
             .option("truncate", False)
             .start()
)

## The assignment starts here

Remark: We consider a day being 5 minutes so we do not have to wait an actual day for the code results. Hence, an hour corresponds to 12.5 seconds.

### Task 2

Hourly gainers

In [5]:
task2_hourly = (
    parsed_df
        .withWatermark("timestamp", "20 seconds")
        .groupBy(window("timestamp", "12.5 seconds"), "name")
        .agg(
            first("price").alias("price_before"),
            last("price").alias("price_after"),
            (last("price") - first("price")).alias("hourly_gain"),
            ((last("price") - first("price")) / first("price") * 100).alias("percentage_gain"),
            count("*").alias("n_events")
        )
        .orderBy(desc("hourly_gain"))
)

task2_hourly_query = (
    task2_hourly.writeStream
        .format("console")
        .outputMode("complete")
        .option("truncate", False)
        .start()
)

25/11/24 20:38:14 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-0e6160bf-3bf7-4311-b4ee-732544c544a4. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------------------------------------------+----+------------+-----------+-----------+---------------+--------+
|window                                      |name|price_before|price_after|hourly_gain|percentage_gain|n_events|
+--------------------------------------------+----+------------+-----------+-----------+---------------+--------+
|[2025-11-24 20:38:07.5, 2025-11-24 20:38:20]|ARNC|36.15       |36.15      |0.0        |0.0            |1       |
|[2025-11-24 20:38:07.5, 2025-11-24 20:38:20]|MDLZ|34.52       |34.52      |0.0        |0.0            |1       |
+--------------------------------------------+----+------------+-----------+-----------+---------------+--------+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------------------------------------------+----+------------+-----------+-----------+------------------+--------+
|window                                      |name|price_before|price_after|hourly_gain|percentage_gain   |n_events|
+--------------------------------------------+----+------------+-----------+-----------+------------------+--------+
|[2025-11-24 20:38:07.5, 2025-11-24 20:38:20]|ILMN|137.07      |155.0      |17.929993  |13.080901523095797|2       |
|[2025-11-24 20:38:07.5, 2025-11-24 20:38:20]|CHRW|51.56       |64.09      |12.529995  |24.301773915565423|2       |
|[2025-11-24 20:38:07.5, 2025-11-24 20:38:20]|GOOG|533.0       |545.32     |12.320007  |2.3114460270579267|2       |
|[2025-11-24 20:38:07.5, 2025-11-24 20:38:20]|SLG |99.65       |110.63     |10.979996  |11.01856052123349 |3       |
|[2025-11-24 20:38:07.5, 2025-11-24 20:38:20]|MTD |242.29      |253.13     |10.84001

-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------------------------------+----+------------+-----------+-----------+------------------+--------+
|window                                      |name|price_before|price_after|hourly_gain|percentage_gain   |n_events|
+--------------------------------------------+----+------------+-----------+-----------+------------------+--------+
|[2025-11-24 20:38:07.5, 2025-11-24 20:38:20]|ILMN|137.07      |155.0      |17.929993  |13.080901523095797|2       |
|[2025-11-24 20:38:20, 2025-11-24 20:38:32.5]|MHK |132.79      |148.535    |15.74501   |11.857075963585281|2       |
|[2025-11-24 20:38:07.5, 2025-11-24 20:38:20]|CHRW|51.56       |64.09      |12.529995  |24.301773915565423|2       |
|[2025-11-24 20:38:07.5, 2025-11-24 20:38:20]|GOOG|533.0       |545.32     |12.320007  |2.3114460270579267|2       |
|[2025-11-24 20:38:07.5, 2025-11-24 20:38:20]|SLG |99.65       |110.63     |10.97999

-------------------------------------------
Batch: 3
-------------------------------------------
+--------------------------------------------+----+------------+-----------+-----------+------------------+--------+
|window                                      |name|price_before|price_after|hourly_gain|percentage_gain   |n_events|
+--------------------------------------------+----+------------+-----------+-----------+------------------+--------+
|[2025-11-24 20:38:07.5, 2025-11-24 20:38:20]|ILMN|137.07      |155.0      |17.929993  |13.080901523095797|2       |
|[2025-11-24 20:38:20, 2025-11-24 20:38:32.5]|MHK |132.79      |148.535    |15.74501   |11.857075963585281|2       |
|[2025-11-24 20:38:07.5, 2025-11-24 20:38:20]|CHRW|51.56       |64.09      |12.529995  |24.301773915565423|2       |
|[2025-11-24 20:38:07.5, 2025-11-24 20:38:20]|GOOG|533.0       |545.32     |12.320007  |2.3114460270579267|2       |
|[2025-11-24 20:38:20, 2025-11-24 20:38:32.5]|GS  |175.27      |186.32     |11.05000

...

-------------------------------------------
Batch: 85
-------------------------------------------
+--------------------------------------------+-----+------------+-----------+-----------+------------------+--------+
|window                                      |name |price_before|price_after|hourly_gain|percentage_gain   |n_events|
+--------------------------------------------+-----+------------+-----------+-----------+------------------+--------+
|[2025-11-24 20:38:57.5, 2025-11-24 20:39:10]|PCLN |1820.46     |2043.95    |223.48999  |12.276567187958486|2       |
|[2025-11-24 20:41:40, 2025-11-24 20:41:52.5]|PCLN |1589.2      |1795.56    |206.3601   |12.985156919348329|3       |
|[2025-11-24 20:40:37.5, 2025-11-24 20:40:50]|PCLN |707.99      |913.82     |205.83002  |29.072447340915826|3       |
|[2025-11-24 20:38:32.5, 2025-11-24 20:38:45]|GOOGL|558.79      |762.2      |203.41003  |36.40187587074689 |2       |
|[2025-11-24 20:40:25, 2025-11-24 20:40:37.5]|GOOG |973.04      |1175.94    

-------------------------------------------
Batch: 86
-------------------------------------------
+--------------------------------------------+-----+------------+-----------+-----------+------------------+--------+
|window                                      |name |price_before|price_after|hourly_gain|percentage_gain   |n_events|
+--------------------------------------------+-----+------------+-----------+-----------+------------------+--------+
|[2025-11-24 20:38:57.5, 2025-11-24 20:39:10]|PCLN |1820.46     |2043.95    |223.48999  |12.276567187958486|2       |
|[2025-11-24 20:41:40, 2025-11-24 20:41:52.5]|PCLN |1589.2      |1795.56    |206.3601   |12.985156919348329|3       |
|[2025-11-24 20:40:37.5, 2025-11-24 20:40:50]|PCLN |707.99      |913.82     |205.83002  |29.072447340915826|3       |
|[2025-11-24 20:38:32.5, 2025-11-24 20:38:45]|GOOGL|558.79      |762.2      |203.41003  |36.40187587074689 |2       |
|[2025-11-24 20:40:25, 2025-11-24 20:40:37.5]|GOOG |973.04      |1175.94    

-------------------------------------------
Batch: 87
-------------------------------------------
+--------------------------------------------+-----+------------+-----------+-----------+------------------+--------+
|window                                      |name |price_before|price_after|hourly_gain|percentage_gain   |n_events|
+--------------------------------------------+-----+------------+-----------+-----------+------------------+--------+
|[2025-11-24 20:38:57.5, 2025-11-24 20:39:10]|PCLN |1820.46     |2043.95    |223.48999  |12.276567187958486|2       |
|[2025-11-24 20:41:40, 2025-11-24 20:41:52.5]|PCLN |1589.2      |1795.56    |206.3601   |12.985156919348329|3       |
|[2025-11-24 20:40:37.5, 2025-11-24 20:40:50]|PCLN |707.99      |913.82     |205.83002  |29.072447340915826|3       |
|[2025-11-24 20:38:32.5, 2025-11-24 20:38:45]|GOOGL|558.79      |762.2      |203.41003  |36.40187587074689 |2       |
|[2025-11-24 20:40:25, 2025-11-24 20:40:37.5]|GOOG |973.04      |1175.94    

-------------------------------------------
Batch: 88
-------------------------------------------
+--------------------------------------------+-----+------------+-----------+-----------+------------------+--------+
|window                                      |name |price_before|price_after|hourly_gain|percentage_gain   |n_events|
+--------------------------------------------+-----+------------+-----------+-----------+------------------+--------+
|[2025-11-24 20:38:57.5, 2025-11-24 20:39:10]|PCLN |1820.46     |2043.95    |223.48999  |12.276567187958486|2       |
|[2025-11-24 20:41:40, 2025-11-24 20:41:52.5]|PCLN |1589.2      |1795.56    |206.3601   |12.985156919348329|3       |
|[2025-11-24 20:40:37.5, 2025-11-24 20:40:50]|PCLN |707.99      |913.82     |205.83002  |29.072447340915826|3       |
|[2025-11-24 20:38:32.5, 2025-11-24 20:38:45]|GOOGL|558.79      |762.2      |203.41003  |36.40187587074689 |2       |
|[2025-11-24 20:40:25, 2025-11-24 20:40:37.5]|GOOG |973.04      |1175.94    

25/11/24 20:42:33 ERROR TaskContextImpl: Error in TaskCompletionListener        
ExitCodeException exitCode=143: 
	at org.apache.hadoop.util.Shell.runCommand(Shell.java:585)
	at org.apache.hadoop.util.Shell.run(Shell.java:482)
	at org.apache.hadoop.util.Shell$ShellCommandExecutor.execute(Shell.java:776)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:869)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:852)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:733)
	at org.apache.hadoop.fs.RawLocalFileSystem$LocalFSFileOutputStream.<init>(RawLocalFileSystem.java:225)
	at org.apache.hadoop.fs.RawLocalFileSystem$LocalFSFileOutputStream.<init>(RawLocalFileSystem.java:209)
	at org.apache.hadoop.fs.RawLocalFileSystem.createOutputStreamWithMode(RawLocalFileSystem.java:307)
	at org.apache.hadoop.fs.RawLocalFileSystem.create(RawLocalFileSystem.java:296)
	at org.apache.hadoop.fs.RawLocalFileSystem.create(RawLocalFileSystem.java:328)
	at org.apache.had